In [ ]:
# Install dependencies
!pip3.9 install --upgrade pip
!pip3.9 install pandas tqdm multiprocess

In [1]:
# from tqdm.notebook import tqdm, trange
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob

In [2]:
datadir = "./data/*/"
all_datas = sorted(glob.glob(datadir + "*.csv"))

In [3]:
# CSV Count
print("Data CSV Count:", len(all_datas))

Data CSV Count: 42863


In [4]:
from multiprocess import Process, Pool

print("Read files...")

def read(filename):
    name = filename
    
    data = pd.read_csv(filename)
    data["filename"] = name
    # print("Read", filename.split("/")[-1])
    return (name, data)

pool = Pool()
result = list(tqdm(pool.imap(read, all_datas), total=len(all_datas)))

pool.close()
pool.join() # wait till pool to finish calculation

print("Finished Reading files...")

all_df = dict(result)

Read files...


 23%|██████▊                       | 9694/42863 [00:08<00:28, 1162.69it/s]Process ForkPoolWorker-1:
Process ForkPoolWorker-4:

Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-5:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, 

  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/pickle.py", line 717, in save_reduce
    save(state)
  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/pickle.py", line 560, in save
    f(self, obj)  # Call unbound method with explicit self
  File "/usr/local/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 586, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "pandas/_libs/parsers.pyx", line 1129, in pandas._libs.parsers.TextReader._convert_with_dtype
  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/pickle.py", line 560, in save
    f(self, obj)  # Call unbound method with explicit self
  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/pickle.py", line 886, in save_tuple
    save(element)
  File "/usr/local/lib/python3.9/site-packages/pandas/io/parsers/reader

  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/pickle.py", line 717, in save_reduce
    save(state)
  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/pickle.py", line 560, in save
    f(self, obj)  # Call unbound method with explicit self
  File "/usr/local/lib/python3.9/site-packages/pandas/core/dtypes/common.py", line 569, in <genexpr>
    return any(is_excluded(dtype) for is_excluded in is_excluded_checks)
  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/pickle.py", line 560, in save
    f(self, obj)  # Call unbound method with explicit self
KeyboardInterrupt
  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/pickle.py", line 886, in save_tuple
    save(element)
  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/pickle.py", line 603, i

KeyboardInterrupt: 

In [ ]:
# 버스 정류장
busstation1 = "PHB351016075" # 조회할 버스 정류장 - PHB351016075 농협양덕지점 (한동대방향)
busstation2 = "PHB351016076" # 조회할 버스 정류장2 (지나갔을까봐) - PHB351016075 농협양덕지점 (한동대방향)

all_datas = pd.concat(all_df.values(), ignore_index=True) # 모든 데이터를 합친다
all_datas = all_datas[(all_datas.nodeid == busstation1) | (all_datas.nodeid == busstation2)] # 정류장 검색

all_datas.reset_index(inplace=True) # Reset index
all_datas = all_datas[["vehicleno", "nodenm", "filename"]] # vehicle 번호와 filename 만 보여주기
all_datas["time"] = pd.to_datetime(all_datas["filename"], format="%y%m%d/%H%M%S", errors="coerce", exact=False) # filename 에서 시간 계산
all_datas.set_index("time", inplace=True)

all_datas.head()

In [ ]:
# 다음 entry 가 같은 vehicleno 일경우 무시
all_datas = all_datas[ 
    all_datas.vehicleno.ne(all_datas.vehicleno.shift())
]
all_datas.head()

In [ ]:
# 빈파일 목록 확인
empty_files = []
for name, data in all_df.items():
    if len(data) == 0:
        empty_files.append(name)
len(empty_files)

In [ ]:
# 빈파일들에는 null 데이터를 추가하여 다음 시간이 계산이 안되게 처리.
for file in empty_files:
    datetime = pd.to_datetime(file, format="%y%m%d/%H%M%S", errors="coerce", exact=False)
    all_datas.loc[datetime] = [np.nan for i in range(len(all_datas.columns))]

In [ ]:
# 마지막으로 전에 빈 공간이 있을경우 데이터 무시.
all_datas = all_datas.sort_index()
all_datas["diff"] = all_datas.index.to_series().diff()
all_datas = all_datas[~all_datas.vehicleno.shift().isna()].dropna(subset=['filename'])

In [ ]:
# 시간 계산
# all_datas["diff"] = all_datas.index.to_series().diff()

In [ ]:
# all_datas["diff"]
diff_data = all_datas.dropna(subset=["diff"])["diff"].dt.seconds.div(60, fill_value=0)
goodcond = (diff_data < 120) & (diff_data > 2)
diff_data[goodcond].describe() # 최대시간을 2시간으로 두어 익일인 경우 무시.

------
# Visuals

In [ ]:
!pip3.9 install matplotlib seaborn

In [ ]:
# Dependencies
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 한글 폰트 설치
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

import platform
if platform.system() == "Darwin":
  rc("font", family="AppleGothic")
elif platform.system() == "Windows":
  path="c:/Windows/Font/malgun.ttf"
  font_name = font_manager.FontPropertie(fname=path).get_name()
  rc('font', family=font_name)
else:
  %config InlineBackend.figure_format = 'retina'
  !apt -qq -y install fonts-nanum
  fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
  font = font_manager.FontProperties(fname=fontpath, size=9)
  plt.rc('font', family='NanumBarunGothic')
  font_manager._rebuild()

In [ ]:
vis = pd.DataFrame(all_datas)
vis['dval'] = all_datas.dropna(subset=["diff"])["diff"].dt.seconds.div(60, fill_value=0)
vis = vis[(goodcond)]
vis.head()

In [ ]:
vis['hour'] = vis.index.hour
hourdata = vis.groupby(by="hour").mean()
hourdata

In [ ]:
%matplotlib inline

# 그래프 준비
plt.figure(figsize=(10,15))
# hourdata.dtypes
# plt.barh(hourdata.index.astype(str), hourdata['dval'])

barg = sns.barplot(x=hourdata['dval'], y=hourdata.index.astype(str), alpha=0.9, orient='h')

# Label 추가
for p in barg.patches:
    width = p.get_width()
    plt.text(1+p.get_width(), p.get_y()+0.55*p.get_height(),
             '{:1.2f}'.format(width),
             ha='center', va='center')

plt.title("302번 농협 양덕지점 (한동대 방향) 시간대별 배차간격")
plt.xlabel("평균 배차간격 (분)")
plt.ylabel("시간대")

plt.show()



In [ ]:
vis['hour'] = vis.index.hour
hourdataMid = vis.groupby(by="hour").median()
hourdataMid

In [ ]:
# 그래프 준비
plt.figure(figsize=(10,15))
# hourdata.dtypes
# plt.barh(hourdata.index.astype(str), hourdata['dval'])

barg = sns.barplot(x=hourdataMid['dval'], y=hourdataMid.index.astype(str), alpha=0.9, orient='h')

# Label 추가
for p in barg.patches:
    width = p.get_width()
    plt.text(1+p.get_width(), p.get_y()+0.55*p.get_height(),
             '{:1.2f}'.format(width),
             ha='center', va='center')

plt.title("302번 농협 양덕지점 (한동대 방향) 시간대별 배차간격")
plt.xlabel("중간값 배차간격 (분)")
plt.ylabel("시간대")

plt.show()

------------

# Test Process

In [ ]:
copy = pd.DataFrame(all_datas)
copy.head()

In [ ]:
copy.dropna(subset=["diff"], inplace=True)
copy['dval'] = all_datas.dropna(subset=["diff"])["diff"].dt.seconds.div(60, fill_value=0)
copy.head()

In [ ]:
copy.loc[copy['dval'] > 70].sort_values(by=["dval"]).head()

In [ ]:
copy.loc[copy['dval'] == 0]

In [ ]:
# all Days
print("Number of days:", len(copy.groupby(pd.Grouper(freq='D'))))

In [ ]:
copy.sort_values(['time', 'vehicleno']).loc["2021-10-18"]

In [ ]:
# 빈파일 목록 확인
empty_files = []
for name, data in all_df.items():
    if len(data) == 0:
        empty_files.append(name)
len(empty_files)

In [ ]:
import numpy as np
# 빈파일들에는 null 데이터를 추가하여 다음 시간이 계산이 안되게 처리.
for file in empty_files:
    datetime = pd.to_datetime(file, format="%y%m%d/%H%M%S", errors="coerce", exact=False)
    copy.loc[datetime] = [np.nan for i in range(len(copy.columns))]

In [ ]:
copyr = copy.sort_index()
copyr["diff2"] = copyr.index.to_series().diff()
copyr = copyr[~copyr.vehicleno.shift().isna()].dropna(subset=['filename'])
print(len(copyr))
# copyr.loc['2021-09-14 13:30':'2021-09-14 16:30']

In [ ]:
# all_datas["diff"]
copyr_diffdata = copyr.dropna(subset=["diff"])["diff"].dt.seconds.div(60, fill_value=0)
copyr_diffdata[(copyr_diffdata < 120) & (copyr_diffdata > 2)].describe() # 최대시간을 2시간으로 두어 익일인 경우 무시.

In [ ]:
# Remove diff data if last row doesn't have any information
for item in copyr.index:
    print(copyr.vehicleno.shift().loc[item])
    if pd.isnull(copyr.vehicleno.shift().loc[item]):
        copyr.loc[item, "dval"] = np.nan

In [ ]:
copyr.diff2

In [ ]:
copy

In [ ]:
copy.sort_index().loc['2021-09-14 13:30':'2021-09-14 16:30']

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
copy.sort_index().loc['2021-09-14']

In [ ]:
# Filter all empty files
len(result)

In [ ]:
# https://stackoverflow.com/questions/56057755/extract-all-rows-in-dataframe-having-same-date
for name, group in copy.groupby(pd.Grouper(freq='D')):
    print(name)